In [2]:
import requests
import feedparser
from bs4 import BeautifulSoup
from scholarly import scholarly

def search_google_scholar(query, num_results=5):
    search_query = scholarly.search_pubs(query)
    results = []
    for i in range(num_results):
        try:
            paper = next(search_query)
            results.append({
                'title': paper['bib']['title'],
                'author': paper['bib'].get('author', 'N/A'),
                'venue': paper['bib'].get('venue', 'N/A'),
                'year': paper['bib'].get('year', 'N/A'),
                'link': paper.get('pub_url', 'N/A')
            })
        except StopIteration:
            break
    return results

def search_arxiv(query, num_results=5):
    base_url = "http://export.arxiv.org/api/query?search_query="
    response = requests.get(f"{base_url}{query.replace(' ', '+')}&max_results={num_results}")
    feed = feedparser.parse(response.text)
    results = []
    for entry in feed.entries:
        results.append({
            'title': entry.title,
            'author': ', '.join(author.name for author in entry.authors),
            'published': entry.published,
            'summary': entry.summary,
            'link': entry.link
        })
    return results

def search_ieee(query, num_results=5):
    search_url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={query.replace(' ', '%20')}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    results = []
    for div in soup.find_all('div', class_='List-results-items')[:num_results]:
        title_tag = div.find('h2', class_='title')
        if title_tag:
            title = title_tag.text.strip()
            link = "https://ieeexplore.ieee.org" + title_tag.find('a')['href']
            results.append({'title': title, 'link': link})
    return results

def search_acm(query, num_results=5):
    search_url = f"https://dl.acm.org/action/doSearch?AllField={query.replace(' ', '+')}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    results = []
    for div in soup.find_all('div', class_='issue-item')[:num_results]:
        title_tag = div.find('h5', class_='issue-item__title')
        if title_tag:
            title = title_tag.text.strip()
            link = "https://dl.acm.org" + title_tag.find('a')['href']
            results.append({'title': title, 'link': link})
    return results

def main():
    query = input("Enter search query: ")

    print("\nGoogle Scholar Results:")
    for paper in search_google_scholar(query):
        print(paper)

    print("\nArXiv Results:")
    for paper in search_arxiv(query):
        print(paper)

    print("\nIEEE Xplore Results:")
    for paper in search_ieee(query):
        print(paper)

    print("\nACM Digital Library Results:")
    for paper in search_acm(query):
        print(paper)

if __name__ == "__main__":
    main()



Google Scholar Results:
{'title': 'Binary code summarization: Benchmarking chatgpt/gpt-4 and other large language models', 'author': ['X Jin', 'J Larson', 'W Yang', 'Z Lin'], 'venue': 'arXiv preprint arXiv:2312.09601', 'year': 'N/A', 'link': 'https://arxiv.org/abs/2312.09601'}
{'title': 'Automatic semantic augmentation of language model prompts (for code summarization)', 'author': ['T Ahmed', 'KS Pai', 'P Devanbu', 'E Barr'], 'venue': 'Proceedings of the IEEE/ACM …', 'year': 'N/A', 'link': 'https://dl.acm.org/doi/abs/10.1145/3597503.3639183'}
{'title': 'Few-shot training LLMs for project-specific code-summarization', 'author': ['T Ahmed', 'P Devanbu'], 'venue': 'Proceedings of the 37th IEEE/ACM International …', 'year': 'N/A', 'link': 'https://dl.acm.org/doi/abs/10.1145/3551349.3559555'}
{'title': 'Source code summarization in the era of large language models', 'author': ['W Sun', 'Y Miao', 'Y Li', 'H Zhang', 'C Fang', 'Y Liu'], 'venue': 'arXiv preprint arXiv …', 'year': 'N/A', 'link'

In [1]:
print('hello')

hello


In [2]:
import requests
import feedparser
from scholarly import scholarly

# --- Google Scholar Search ---
def search_google_scholar(keywords, num_results=5):
    combined_query = " OR ".join(keywords)  # Combine keywords with OR
    search_query = scholarly.search_pubs(combined_query)

    results = []
    for i in range(num_results):
        try:
            paper = next(search_query)
            results.append({
                "title": paper.get("bib", {}).get("title", "N/A"),
                "authors": paper.get("bib", {}).get("author", "N/A"),
                "year": paper.get("bib", {}).get("pub_year", "N/A"),
                "link": f'https://scholar.google.com/scholar_lookup?title={paper.get("bib", {}).get("title", "").replace(" ", "+")}'
            })
        except StopIteration:
            break
    return results

# --- arXiv API Search ---
def search_arxiv(keywords, num_results=5):
    combined_query = " OR ".join(keywords)  # Use OR between keywords
    base_url = "http://export.arxiv.org/api/query"
    params = {"search_query": f"all:{combined_query}", "start": 0, "max_results": num_results}
    response = requests.get(base_url, params=params)
    feed = feedparser.parse(response.text)

    results = []
    for entry in feed.entries:
        results.append({
            "title": entry.title,
            "authors": [author.name for author in entry.authors],
            "published": entry.published,
            "link": entry.link
        })
    return results

# --- Reverted IEEE Xplore Search (Direct Search Links) ---
def search_ieee_explore(keywords):
    results = []
    for keyword in keywords:  # Search each keyword separately
        search_url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={keyword.replace(' ', '+')}"
        results.append({"IEEE Search": search_url})
    return results

# --- Reverted ACM Digital Library Search (Direct Search Links) ---
def search_acm_digital_library(keywords):
    results = []
    for keyword in keywords:  # Search each keyword separately
        search_url = f"https://dl.acm.org/action/doSearch?AllField={keyword.replace(' ', '+')}"
        results.append({"ACM Search": search_url})
    return results

# --- 🔥 Perform the Search ---
def search_papers_with_or_logic(keywords):
    print(f"\n🔹 **Searching for papers with keywords: {', '.join(keywords)} (OR logic for Google Scholar & arXiv, direct links for IEEE & ACM)**")

    print("\n📌 Google Scholar Results:")
    for paper in search_google_scholar(keywords):
        print(paper)

    print("\n📌 arXiv Results:")
    for paper in search_arxiv(keywords):
        print(paper)

    print("\n📌 IEEE Xplore Search Links:")
    for paper in search_ieee_explore(keywords):
        print(paper)

    print("\n📌 ACM Digital Library Search Links:")
    for paper in search_acm_digital_library(keywords):
        print(paper)


# --- Run the Search ---
keywords = ["benchmark dataset code summarization", "benchmark dataset code generation"]
search_papers_with_or_logic(keywords)



🔹 **Searching for papers with keywords: benchmark dataset code summarization, benchmark dataset code generation (OR logic for Google Scholar & arXiv, direct links for IEEE & ACM)**

📌 Google Scholar Results:
{'title': 'Codexglue: A machine learning benchmark dataset for code understanding and generation', 'authors': ['S Lu', 'D Guo', 'S Ren', 'J Huang', 'A Svyatkovskiy'], 'year': '2021', 'link': 'https://scholar.google.com/scholar_lookup?title=Codexglue:+A+machine+learning+benchmark+dataset+for+code+understanding+and+generation'}
{'title': 'Aixbench: A code generation benchmark dataset', 'authors': ['Y Hao', 'G Li', 'Y Liu', 'X Miao', 'H Zong', 'S Jiang'], 'year': '2022', 'link': 'https://scholar.google.com/scholar_lookup?title=Aixbench:+A+code+generation+benchmark+dataset'}
{'title': 'Codereval: A benchmark of pragmatic code generation with generative pre-trained models', 'authors': ['H Yu', 'B Shen', 'D Ran', 'J Zhang', 'Q Zhang', 'Y Ma'], 'year': '2024', 'link': 'https://scholar.go

In [5]:
import requests
import feedparser
from scholarly import scholarly

# --- Google Scholar Search (Filter by Year) ---
def search_google_scholar(keywords, num_results=5):
    combined_query = " OR ".join(keywords)  # Combine keywords with OR
    search_query = scholarly.search_pubs(combined_query)

    results = []
    for i in range(num_results * 2):  # Fetch extra results to filter later
        try:
            paper = next(search_query)
            year = int(paper.get("bib", {}).get("pub_year", 0))
            if year >= 2021:  # Only keep papers published after 2020
                results.append({
                    "title": paper.get("bib", {}).get("title", "N/A"),
                    "authors": paper.get("bib", {}).get("author", "N/A"),
                    "year": year,
                    "link": f'https://scholar.google.com/scholar_lookup?title={paper.get("bib", {}).get("title", "").replace(" ", "+")}'
                })
            if len(results) >= num_results:
                break
        except (StopIteration, ValueError):
            break
    return results

# --- arXiv API Search (Filter by Year) ---
def search_arxiv(keywords, num_results=5):
    combined_query = " OR ".join(keywords)  # Use OR between keywords
    base_url = "http://export.arxiv.org/api/query"
    params = {"search_query": f"all:{combined_query}", "start": 0, "max_results": num_results * 2}
    response = requests.get(base_url, params=params)
    feed = feedparser.parse(response.text)

    results = []
    for entry in feed.entries:
        year = int(entry.published[:4])  # Extract year from published date
        if year >= 2021:  # Only keep papers published after 2020
            results.append({
                "title": entry.title,
                "authors": [author.name for author in entry.authors],
                "year": year,
                "link": entry.link
            })
        if len(results) >= num_results:
            break
    return results

# --- 🔥 Perform the Search ---
def search_papers_with_year_filter(keywords):
    print(f"\n🔹 **Searching for papers with keywords: {', '.join(keywords)} (Published after 2020)**")

    print("\n📌 Google Scholar Results:")
    for paper in search_google_scholar(keywords, 1000):
        print(paper)

    #print("\n📌 arXiv Results:")
    #for paper in search_arxiv(keywords):
        #print(paper)

# --- Run the Search ---
keywords = ["benchmark code summarization", "benchmark code generation"]
search_papers_with_year_filter(keywords)



🔹 **Searching for papers with keywords: benchmark code summarization, benchmark code generation (Published after 2020)**

📌 Google Scholar Results:
{'title': 'Codereval: A benchmark of pragmatic code generation with generative pre-trained models', 'authors': ['H Yu', 'B Shen', 'D Ran', 'J Zhang', 'Q Zhang', 'Y Ma'], 'year': 2024, 'link': 'https://scholar.google.com/scholar_lookup?title=Codereval:+A+benchmark+of+pragmatic+code+generation+with+generative+pre-trained+models'}
{'title': 'Codexglue: A machine learning benchmark dataset for code understanding and generation', 'authors': ['S Lu', 'D Guo', 'S Ren', 'J Huang', 'A Svyatkovskiy'], 'year': 2021, 'link': 'https://scholar.google.com/scholar_lookup?title=Codexglue:+A+machine+learning+benchmark+dataset+for+code+understanding+and+generation'}
{'title': 'Classeval: A manually-crafted benchmark for evaluating llms on class-level code generation', 'authors': ['X Du', 'M Liu', 'K Wang', 'H Wang', 'J Liu', 'Y Chen'], 'year': 2023, 'link': 

In [ ]:
import requests
import feedparser
from scholarly import scholarly

# --- Google Scholar Search (Filter by Year) ---
def search_google_scholar(keywords, num_results=5):
    combined_query = " OR ".join(keywords)  # Combine keywords with OR
    search_query = scholarly.search_pubs(combined_query)

    results = []
    for i in range(num_results * 2):  # Fetch extra results to filter later
        try:
            paper = next(search_query)
            year = int(paper.get("bib", {}).get("pub_year", 0))
            if year >= 2021:  # Only keep papers published after 2020
                results.append({
                    "title": paper.get("bib", {}).get("title", "N/A"),
                    "authors": paper.get("bib", {}).get("author", "N/A"),
                    "year": year,
                    "link": f'https://scholar.google.com/scholar_lookup?title={paper.get("bib", {}).get("title", "").replace(" ", "+")}'
                })
            if len(results) >= num_results:
                break
        except (StopIteration, ValueError):
            break
    return results

# --- arXiv API Search (Filter by Year) ---
def search_arxiv(keywords, num_results=5):
    combined_query = " OR ".join(keywords)  # Use OR between keywords
    base_url = "http://export.arxiv.org/api/query"
    params = {"search_query": f"all:{combined_query}", "start": 0, "max_results": num_results * 2}
    response = requests.get(base_url, params=params)
    feed = feedparser.parse(response.text)

    results = []
    for entry in feed.entries:
        year = int(entry.published[:4])  # Extract year from published date
        if year >= 2021:  # Only keep papers published after 2020
            results.append({
                "title": entry.title,
                "authors": [author.name for author in entry.authors],
                "year": year,
                "link": entry.link
            })
        if len(results) >= num_results:
            break
    return results

# --- 🔥 Perform the Search ---
def search_papers_with_year_filter(keywords):
    print(f"\n🔹 **Searching for papers with keywords: {', '.join(keywords)} (Published after 2020)**")

    print("\n📌 Google Scholar Results:")
    for paper in search_google_scholar(keywords, 1000):
        print(paper)

    #print("\n📌 arXiv Results:")
    #for paper in search_arxiv(keywords):
        #print(paper)

# --- Run the Search ---
keywords = ["benchmark code summarization", "benchmark code generation", "benchmark test case generation", "benchmark requirements generation", "benchmark code optimization", "benchmark code "]
search_papers_with_year_filter(keywords)



🔹 **Searching for papers with keywords: benchmark code summarization, benchmark code generation, benchmark test case generation, benchmark requirements generation, benchmark code optimization, benchmark code translation (Published after 2020)**

📌 Google Scholar Results:
{'title': 'Codexglue: A machine learning benchmark dataset for code understanding and generation', 'authors': ['S Lu', 'D Guo', 'S Ren', 'J Huang', 'A Svyatkovskiy'], 'year': 2021, 'link': 'https://scholar.google.com/scholar_lookup?title=Codexglue:+A+machine+learning+benchmark+dataset+for+code+understanding+and+generation'}
{'title': 'A survey on evaluating large language models in code generation tasks', 'authors': ['L Chen', 'Q Guo', 'H Jia', 'Z Zeng', 'X Wang', 'Y Xu'], 'year': 2024, 'link': 'https://scholar.google.com/scholar_lookup?title=A+survey+on+evaluating+large+language+models+in+code+generation+tasks'}
{'title': 'JavaBench: A Benchmark of Object-Oriented Code Generation for Evaluating Large Language Models'

In [9]:
all_papers

[]

In [4]:
import random
# driver=webdriver.Chrome()
# driver.get("https://www.facebook.com/")
from selenium.webdriver.common.by import By
# from util import io
from urllib.parse import urlencode, urlparse, parse_qs
from lxml.html import fromstring
from requests import get
import urllib.request
from xml.etree.ElementTree import XML, fromstring
import time
import csv
import re
import json
from selenium import webdriver 
import requests
import bs4
import codecs

driver=webdriver.Chrome()

driver.get("https://www.facebook.com/")



def parse_selenium_text(whole_text, keyword):
  new_list = []
  lo = whole_text.splitlines()
  ss = keyword.split('+')
  ss = '|'.join(ss)
  for item in lo:
    if re.search(r'('+ss+')', item):
        new_list.append([item, 'sciencedirect'])
  return new_list

def write_to_csv(paper_list, filename):
  with open(filename+".csv", "a", newline="") as f:
      writer = csv.writer(f, dialect='excel', delimiter='\n')
      writer.writerow(paper_list)

def parse_title(title_list):
  string_list = []
  for item in title_list:
    if isinstance(item, bs4.element.Tag):
      string_list.append(item.contents[0])
    if isinstance(item, bs4.element.NavigableString):
      string_list.append(item)
  string_list = ' '.join(string_list)
  return string_list



def newWriter(paper_list, filename):
  result_file = open(filename+".csv",'a', newline='', encoding="utf-8")
  wr = csv.writer(result_file, dialect='excel')
  #wr.writerows([[item] for item in paper_list])
  wr.writerow([paper_list[0], paper_list[1]])


def parse_acm(pager_number, myurl):
  paper_list = []
  content = requests.get(myurl)

  page_soup = soup(content.text, "html.parser")

  current_page = page_soup.contents[2].contents[2].contents[9].contents[1].contents[3].contents[1].contents[0].contents[1].contents[3].contents[3].contents
  if len(current_page) == 2:
    return None
  for item in current_page:
    if isinstance(item, bs4.element.Tag):
      if bool(item.attrs) == False:
        continue 
      if item.attrs['class'][0] == 'search__item':
        try:
          current_paper = item.contents[3].contents[3].contents[1].contents
          raw_title_info = current_paper[1].contents[0].contents[0].contents
          doi = current_paper[5].contents[0].attrs['href']
          pub_place = current_paper[5].contents[0].attrs['title']
          title = parse_title(raw_title_info)
          print(title)
          paper_list.append([title, pub_place])
        except:
          print('Parse Error!')

  write_to_csv(paper_list, 'ACMPaperList')



def parse_scienceDirect(init_pgsize, pager_number, myurl, keyword):
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
 
  content = requests.get(myurl, headers=headers)
  driver.get(myurl)
  
  time.sleep(2)
  bidy = driver.find_element(By.CLASS_NAME, 'col-xs-24')
  a = bidy.find_element(By.ID, 'srp-results-list')
  b = a.find_element(By.CLASS_NAME, 'search-result-wrapper')
  print(b)
  
  list = parse_selenium_text(b.text, keyword)

  for item in list:
    newWriter(item, 'scienceDirectPaperList')

  myurl = 'https://www.sciencedirect.com/search?qs='+ keyword +'&date=2010-2021&offset='+ str(init_pgsize)
  print('Analysis of page {} finished'.format(pager_number))
  print('Total number of papers extracted so far:', len(list))
  pager_number += 1
  init_pgsize += 100
  parse_scienceDirect(init_pgsize, pager_number, myurl, keyword)


def parseIEEE(pager_number, pgsize, init_url, keyword):
  print('I am analyzing this page:', pgsize)

  llist = []

  driver.get(init_url)

  time.sleep(2)
  bidy = driver.find_element(By.CLASS_NAME,'main-section')
  a = bidy.find_element(By.TAG_NAME,'xpl-results-list')
  time.sleep(5)
  b = a.find_element(By.CLASS_NAME,'List-results-items')

  for item in b:
    splited = item.text.split('\n')
    llist.append([splited[0], splited[2]])

  for item in llist:
    newWriter(item, 'IEEEpaperslist')

  myurl = 'https://ieeexplore.ieee.org/search/searchresult.jsp?queryText='+keyword+'&highlight=true&returnType=SEARCH&matchPubs=true&ranges=2010_2021_Year&returnFacets=ALL&rowsPerPage='+str(pgsize)+'&pageNumber='+str(pager_number)
  print('Analysis of page {} finished'.format(pager_number))
  print('Total number of papers extracted so far:', len(llist))
  pager_number += 1
  pgsize += 100
  parseIEEE(pager_number, pgsize, myurl, keyword)


def main():
  pager_number = 1
  init_pgsize = 100

  queries = ['benchmark dataset code generation']
  for title in queries:
      # acm_url = 'https://dl.acm.org/action/doSearch?AllField='+title+'&pageSize='+str(init_pgsize)+'&startPage=1'
      # acm_url = f"https://dl.acm.org/action/doSearch?AllField={title}&pageSize={init_pgsize}&startPage={pager_number}"
      # parse_acm(pager_number, acm_url)

      # sd_url = 'https://www.sciencedirect.com/search?qs='+title+'&date=2022-2024'
      # parse_scienceDirect(init_pgsize, pager_number, sd_url, title)

      ieee_link = f'https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={title}&highlight=true&returnType=SEARCH&matchPubs=true&ranges=2022_2024_Year&returnFacets=ALL&rowsPerPage=100&pageNumber=1'
      parseIEEE(pager_number, init_pgsize, ieee_link, title)


if __name__ == '__main__':
    main()

I am analyzing this page: 100


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".main-section"}
  (Session info: chrome=133.0.6943.98); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x61aaf7f14bba <unknown>
#1 0x61aaf79b2790 <unknown>
#2 0x61aaf7a03c80 <unknown>
#3 0x61aaf7a03e01 <unknown>
#4 0x61aaf7a52944 <unknown>
#5 0x61aaf7a29a7d <unknown>
#6 0x61aaf7a4fccc <unknown>
#7 0x61aaf7a29823 <unknown>
#8 0x61aaf79f5a88 <unknown>
#9 0x61aaf79f6bf1 <unknown>
#10 0x61aaf7ede15b <unknown>
#11 0x61aaf7ee20e2 <unknown>
#12 0x61aaf7ecb01c <unknown>
#13 0x61aaf7ee2cd4 <unknown>
#14 0x61aaf7eaf48f <unknown>
#15 0x61aaf7f034f8 <unknown>
#16 0x61aaf7f036c9 <unknown>
#17 0x61aaf7f13a36 <unknown>
#18 0x76bea0a9caa4 <unknown>
#19 0x76bea0b29c3c <unknown>


In [ ]:
import bs4
import codecs
from bs4 import BeautifulSoup as soup
import requests
from timeit import default_timer as timer
import time
import csv
import re
import json
from selenium import webdriver 
import random
driver=webdriver.Chrome()
from selenium.webdriver.common.by import By

def loadtxt():
  q = []
  with open("query.txt", "r") as f:
    q = [x.strip() for x in f.readlines()]
  return q

def jsonWriter(file, filename):
  with open(filename+'.json', 'a') as f:
      json.dump(file, f, ensure_ascii=False)

def newWriter(paper_list, filename):
  result_file = open(filename+".csv",'a', newline='', encoding="utf-8")
  wr = csv.writer(result_file, dialect='excel')
  wr.writerow([paper_list[0], paper_list[1]])

def write_to_csv(paper_list, filename):
  with open(filename+".csv", "a", newline="") as f:
      writer = csv.writer(f, delimiter='\n')
      writer.writerow(paper_list)

def parse_title(title_list):
  string_list = []
  for item in title_list:
    if isinstance(item, bs4.element.Tag):
      string_list.append(item.contents[0])
    if isinstance(item, bs4.element.NavigableString):
      string_list.append(item)
  string_list = ' '.join(string_list)
  return string_list

def parse_selenium_text(whole_text):
  new_list = []
  lo = whole_text.splitlines()
  for item in lo:
    if re.search(r'(vulnerability)', item):
      new_list.append([item, 'sciencedirect'])
  return new_list

def randomize_user_agent():
  user_agent_list = [
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
  'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
  ]

  for i in range(4):
    user_agent = random.choice(user_agent_list)
  
  headers = {'User-Agent': user_agent}
  return headers

def parseIEEE(pager_number, pgsize, init_url, keyword):
  print('I am analyzing this page:', pgsize)

  llist = []

  driver.get(init_url)

  time.sleep(2)
  # bidy = driver.find_element_by_class_name('main-section')
  # a = bidy.find_element_by_tag_name('xpl-results-list')
  # time.sleep(5)
  # b = a.find_elements_by_class_name('List-results-items')
  
  # bidy = driver.find_element(By.CLASS_NAME,'col')
  a = driver.find_element(By.ID,'xplMainContent')
  b = a.find_element(By.TAG_NAME,'xpl-results-list')
  #llist = parse_selenium_text(a.text)
  b = re.split(r'[\n\r]+', a.text)
  for item in b:
    if re.findall(r"(vulnerability|Vulnerability|bug|defect|detection|Detection)", item):
    #splited = item.text.split('\n')
      llist.append([item, 'IEEE'])

  for item in llist:
    newWriter(item, 'IEEEpaperslist')

  myurl = 'https://ieeexplore.ieee.org/search/searchresult.jsp?queryText='+keyword+'&highlight=true&returnType=SEARCH&matchPubs=true&ranges=2022_2024_Year&returnFacets=ALL&rowsPerPage='+str(pgsize)+'&pageNumber='+str(pager_number)
  print('Analysis of page {} finished'.format(pager_number))
  print('Total number of papers extracted so far:', len(llist))
  pager_number += 1
  # pgsize += 100
  parseIEEE(pager_number, pgsize, myurl, keyword)


def googleScholarScraper(pager_number, scholar_pgsize, init_url):
#   print('I am analyzing this page:', scholar_pgsize)
  current_header = randomize_user_agent()
  llist = []
  time.sleep(2)
  content = requests.get(init_url, headers=current_header)

  page_soup = soup(content.text, "html.parser")

  try:
    paper_list = page_soup.contents[1].contents[1].contents[0].contents[13].contents[1].contents[2].contents[1]
    for item in paper_list.contents:
      if isinstance(item, bs4.element.Tag):
        if len(item.contents) > 1:
          if len(item.contents[1].contents[0].contents) == 3:
            for i, sub_item in enumerate(item.contents[1].contents[0].contents[2].contents):
              if isinstance(sub_item, bs4.element.Tag):
                item.contents[1].contents[0].contents[2].contents[i] = sub_item.contents[0]
            _ttl = ' '.join(item.contents[1].contents[0].contents[2].contents)
            # print(_ttl)
            llist.append([_ttl, item.contents[1].contents[1].contents[-1]])
          else:
            if len(item.contents[1].contents[0].contents[0].contents) == 1:
              if isinstance(item.contents[1].contents[0].contents[0].contents[0], bs4.element.Tag):
                llist.append(item.contents[1].contents[0].contents[0].contents[0].contents[0])
                # print(item.contents[1].contents[0].contents[0].contents[0].contents[0])
              else:
                llist.append(item.contents[1].contents[0].contents[0].contents[0])
                # print(item.contents[1].contents[0].contents[0].contents[0])
            if len(item.contents[1].contents[0].contents[0].contents) > 1:
              for i, sub_item in enumerate(item.contents[1].contents[0].contents[0].contents):
                if isinstance(sub_item, bs4.element.Tag):
                  item.contents[1].contents[0].contents[0].contents[i] = sub_item.contents[0]
              _ttl = ' '.join(item.contents[1].contents[0].contents[0].contents)
              # print(_ttl)
              llist.append([_ttl, item.contents[1].contents[1].contents[- 1]])
              
    write_to_csv(llist, 'scholar3')

    # print('Analysis of page {} finished'.format(pager_number))
    # print('Total number of papers extracted so far:', len(paper_list))
  except Exception as e:
    print(e)
  pager_number += 1
  scholar_pgsize += 10

  next_page = 'https://scholar.google.com/scholar?start='+str(scholar_pgsize)+'&q=vulnerability+detection+on+source+code+using+deep+learning&hl=en&as_sdt=0,5&as_ylo=2010&as_yhi=2021'
  
  #time.sleep((130-5)*np.random.random()+5)
  googleScholarScraper(pager_number, scholar_pgsize, next_page)
      

def parse_scienceDirect(init_pgsize, pager_number, myurl, keyword):
  
  paper_list = []
  
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
 
  content = requests.get(myurl, headers=headers)
  driver.get(myurl)
  
  time.sleep(2)
  bidy = driver.find_element(By.CLASS_NAME, 'col-xs-24')
  # bidy = driver.find_elements_by_class_name('col-xs-24')
  a = bidy.find_element(By.ID, 'srp-results-list')
  b = a.find_element(By.CLASS_NAME, 'search-result-wrapper')
  #a = bidy.find_elements_by_id('srp-results-list')
  #b = a[0].find_element_by_class_name('search-result-wrapper')
  
  list = parse_selenium_text(b.text)

  for item in list:
    newWriter(item, 'scienceDirectPaperList')

  myurl = 'https://www.sciencedirect.com/search?qs='+ keyword +'&date=2010-2021&offset='+ str(init_pgsize)
  print('Analysis of page {} finished'.format(pager_number))
  print('Total number of papers extracted so far:', len(list))
  pager_number += 1
  init_pgsize += 100
  parse_scienceDirect(init_pgsize, pager_number, myurl, keyword)


def parse_acm(pager_number, myurl):
  paper_list = []
  content = requests.get(myurl)

  page_soup = soup(content.text, "html.parser")
 
  current_page = page_soup.contents[2].contents[2].contents[9].contents[1].contents[3].contents[1].contents[0].contents[1].contents[3].contents[3]
  # current_page = page_soup.contents[2].contents[2].contents[9].contents[1].contents[3].contents[1].contents[0].contents[1].contents[3].contents[1].contents
  if len(current_page) == 2:
    return None
  for item in current_page:
    if isinstance(item, bs4.element.Tag):
      if bool(item.attrs) == False:
        continue 
      if item.attrs['class'][0] == 'search__item':
        try:
          110
          # current_paper = item.contents[3].contents[3].contents[1].contents
          # raw_title_info = current_paper[1].contents[0].contents[0].contents
          # doi = current_paper[5].contents[0].attrs['href']
          # pub_place = current_paper[5].contents[0].attrs['title']
          title = item.contents[1].contents[1].contents[0].contents[0].text
          # title = parse_title(title)
          detail = item.contents[1].contents[1].contents[0].contents[2].text
          print(title)
          paper_list.append([title, detail])
        except:
          print('Parse Error!')

  write_to_csv(paper_list, 'ACMPaperList')

  if pager_number == 1:
    myurl = page_soup.contents[2].contents[2].contents[9].contents[1].contents[3].contents[1].contents[0].contents[1].contents[3].contents[4].contents[2].contents[0].attrs['href']
  else:
    myurl = page_soup.contents[2].contents[2].contents[9].contents[1].contents[3].contents[1].contents[0].contents[1].contents[3].contents[4].contents[2].contents[0].attrs['href']
  
  print('Analysis of page {} finished'.format(pager_number))
  print('Total number of papers extracted so far:', len(paper_list))
  pager_number += 1
  parse_acm(pager_number, myurl)

def parseIEEErecursive(pager_number, myurl):
  paper_list = []
  content = requests.get(myurl)

  page_soup = soup(content.text, "html.parser")
  current_page = page_soup.contents[2].contents[2].contents[9].contents[1].contents[3].contents[1].contents[0].contents[1].contents[3].contents[3]
  parseIEEErecursive()


def parseIEEEBs4(pageNumber, myurl):
  content = requests.get(myurl)

  page_soup = soup(content.text, "html.parser")
  print('')

def main():
  start = timer()
  pager_number = 1
  init_pgsize = 100
  scholar_pgsize = 450

  # queries = loadtxt()
  queries = [
            # 'Vulnerability+detection',
            'Deep+Transfer+Learning+Vulnerability+Detection',
            'Transfer+Learning+Software+Vulnerability+Detection',
            'Transfer+Learning+Software+Bug+Detection',
            # 'Software+vulnerability+detection',
            # 'Vulnerability+detection+using+deep+learning',
            # 'Source+code+security+bug+prediction',
            # 'Source+code+vulnerability+detection',
            # 'Source+code+bug+detection',
            # 'Vulnerability+detection+on+source+code+using+deep+learning'
    ]
  for title in queries:
    print(title)
    init_url = 'https://scholar.google.com/scholar?q='+title+'&hl=en&as_sdt=0%2C5&as_ylo=2011&as_yhi=2024'
    googleScholarScraper(pager_number, scholar_pgsize, init_url)

    acm_url = 'https://dl.acm.org/action/doSearch?AllField='+title+'&pageSize=100&startPage=1'
    parse_acm(pager_number, acm_url)

    sd_url = 'https://www.sciencedirect.com/search?qs='+title+'&date=2010-2022'
    parse_scienceDirect(init_pgsize, pager_number, sd_url, title)

    ieee_link = 'https://ieeexplore.ieee.org/search/searchresult.jsp?queryText='+title+'&highlight=true&returnType=SEARCH&matchPubs=true&ranges=2022_2024_Year&returnFacets=ALL&rowsPerPage=100&pageNumber=1'
    parseIEEE(pager_number, init_pgsize, ieee_link, title)
    parseIEEEBs4(pager_number, ieee_link)
  end = timer()
  print(end-start)

if __name__ == '__main__':
    main()

Deep+Transfer+Learning+Vulnerability+Detection
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'
'NavigableString' object has no attribute 'contents'

KeyboardInterrupt: 

In [3]:
import bs4
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
import re
import json
from selenium import webdriver
import random

driver = webdriver.Chrome()

def loadtxt():
    return [line.strip() for line in open("query.txt", "r")]

def jsonWriter(data, filename):
    with open(f'{filename}.json', 'a') as f:
        json.dump(data, f, ensure_ascii=False)

def newWriter(row, filename):
    with open(f'{filename}.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)

def randomize_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    ]
    return {'User-Agent': random.choice(user_agents)}

def handle_cookie_consent():
    try:
        cookie_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept') or contains(., 'Agree')]"))
        )
        cookie_btn.click()
        time.sleep(1)
    except TimeoutException:
        pass

def google_scholar_scraper(query, start=0):
    base_url = f"https://scholar.google.com/scholar?start={start}&q={query}&hl=en&as_sdt=0,5"
    driver.get(base_url)
    time.sleep(20)
    time.sleep(random.uniform(2,4))
    
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "gs_ri"))
        )
        
        papers = []
        for result in results:
            try:
                title = result.find_element(By.TAG_NAME, 'h3').text
                link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
                papers.append([title, link])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'GoogleScholarPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No results found or page didn't load")
        return 0

def parse_ieee(keyword, page=1):
    url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={keyword}&pageNumber={page}"
    driver.get(url)
    time.sleep(3)
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".List-results-items"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, "h3 a").text
                year = item.find_element(By.XPATH, ".//*[contains(text(), 'Year:')]").text.split()[-1]
                papers.append([title, f"IEEE {year}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'IEEEPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No IEEE results found")
        return 0

def parse_science_direct(keyword, offset=0):
    url = f"https://www.sciencedirect.com/search?qs={keyword}&offset={offset}"
    driver.get(url)
    time.sleep(2)
    handle_cookie_consent()
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".result-list-content"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, ".result-item-title").text
                details = item.find_element(By.CSS_SELECTOR, ".result-item-subtype").text
                papers.append([title, f"ScienceDirect: {details}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'ScienceDirectPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No ScienceDirect results found")
        return 0

def parse_acm(query, page=1):
    url = f"https://dl.acm.org/action/doSearch?AllField={query}&startPage={page-1}&pageSize=50"
    response = requests.get(url, headers=randomize_user_agent())
    soup = BeautifulSoup(response.text, 'html.parser')
    
    papers = []
    for item in soup.select('.issue-item__title'):
        try:
            title = item.text.strip()
            link = "https://dl.acm.org" + item.find('a')['href']
            papers.append([title, link])
        except:
            continue
    
    for paper in papers:
        newWriter(paper, 'ACMPapers')
    
    return len(papers)

def main():
    queries = [
        'Benchmark+Dataset+Code+Generation+LLM',
        'Benchmark+Dataset+Code+Summarization+LLM',
        'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
        'Benchmark+Dataset+Patch+Generation+LLM',
        'Benchmark+Dataset+Code+Optimization+LLM',
        'Benchmark+Dataset+Code+Translation+LLM',
        'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
        'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
        ]
    
    for query in queries:
        print(f"Processing: {query}")
        
        # Google Scholar (needs careful handling)
        for page in range(0, 15):  # First 5 pages
            count = google_scholar_scraper(query, start=page*10)
            if count == 0: break
        
        # IEEE
        for page in range(1, 15):  # First 5 pages
            count = parse_ieee(query, page)
            if count < 10: break
        
        # ScienceDirect
        #for offset in range(0, 500, 100):  # First 5 pages
            #count = parse_science_direct(query, offset)
            #if count < 20: break
        
        # ACM
        for page in range(1, 15):  # First 5 pages
            count = parse_acm(query, page)
            if count < 10: break
    
    driver.quit()

if __name__ == '__main__':
    main()

Processing: Benchmark+Dataset+Code+Generation+LLM
No IEEE results found
Processing: Benchmark+Dataset+Code+Summarization+LLM
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
No IEEE results found
Processing: Benchmark+Dataset+Patch+Generation+LLM
Processing: Benchmark+Dataset+Code+Optimization+LLM
Processing: Benchmark+Dataset+Code+Translation+LLM
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
No IEEE results found
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
No IEEE results found


In [12]:
import bs4
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
import re
import json
from selenium import webdriver
import random
from datetime import datetime

driver = webdriver.Chrome()

def loadtxt():
    return [line.strip() for line in open("query.txt", "r")]

def jsonWriter(data, filename):
    with open(f'{filename}.json', 'a') as f:
        json.dump(data, f, ensure_ascii=False)

def newWriter(row, filename):
    with open(f'{filename}.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)

def randomize_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    ]
    return {'User-Agent': random.choice(user_agents)}

def handle_cookie_consent():
    try:
        cookie_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept') or contains(., 'Agree')]"))
        )
        cookie_btn.click()
        time.sleep(1)
    except TimeoutException:
        pass

def is_recent_publication(year_text):
    try:
        year = int(re.search(r'\d{4}', year_text).group())
        return year >= 2020
    except (AttributeError, ValueError):
        return False

def google_scholar_scraper(query, start=0):
    base_url = f"https://scholar.google.com/scholar?start={start}&q={query}&hl=en&as_sdt=0,5&as_ylo=2020"
    driver.get(base_url)
    time.sleep(20)

    time.sleep(random.uniform(2,4))
    
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "gs_ri"))
        )
        
        papers = []
        for result in results:
            try:
                title = result.find_element(By.TAG_NAME, 'h3').text
                link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
                year_text = result.find_element(By.CLASS_NAME, 'gs_a').text
                
                if is_recent_publication(year_text):
                    papers.append([title, link, year_text])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'GoogleScholarPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No results found or page didn't load")
        return 0

def parse_ieee(keyword, page=1):
    url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={keyword}&pageNumber={page}&ranges=2020_{datetime.now().year}_Year"
    driver.get(url)
    time.sleep(3)
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".List-results-items"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, "h3 a").text
                year_text = item.find_element(By.XPATH, ".//*[contains(text(), 'Year:')]").text
                
                if is_recent_publication(year_text):
                    papers.append([title, f"IEEE {year_text}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'IEEEPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No IEEE results found")
        return 0

def parse_science_direct(keyword, offset=0):
    url = f"https://www.sciencedirect.com/search?qs={keyword}&offset={offset}&date=2020-{datetime.now().year}"
    driver.get(url)
    time.sleep(2)
    handle_cookie_consent()
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".result-list-content"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, ".result-item-title").text
                year_text = item.find_element(By.CSS_SELECTOR, ".subType").text
                
                if is_recent_publication(year_text):
                    papers.append([title, f"ScienceDirect: {year_text}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'ScienceDirectPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No ScienceDirect results found")
        return 0

def parse_acm(query, page=1):
    url = f"https://dl.acm.org/action/doSearch?AllField={query}&startPage={page-1}&pageSize=50&AfterYear=2020"
    response = requests.get(url, headers=randomize_user_agent())
    soup = BeautifulSoup(response.text, 'html.parser')
    
    papers = []
    for item in soup.select('.issue-item__title'):
        try:
            title = item.text.strip()
            link = "https://dl.acm.org" + item.find('a')['href']
            year_text = item.find_next(class_='bookPubDate').text
            
            if is_recent_publication(year_text):
                papers.append([title, link, year_text])
        except:
            continue
    
    for paper in papers:
        newWriter(paper, 'ACMPapers')
    
    return len(papers)

def main():
    queries = [
        'Benchmark+Dataset+Program+Repair+LLM',
        'Benchmark+Dataset+Requirement+Generation+LLM',
        
    ]
    
    for query in queries:
        print(f"Processing: {query}")
        
        # Google Scholar
        #for page in range(0, 15):  # First 5 pages
            #count = google_scholar_scraper(query, start=page*10)
            #if count == 0: break
        
        # IEEE
        #for page in range(1, 16):  # First 5 pages
            #count = parse_ieee(query, page)
            #if count < 10: break
        
        # ScienceDirect
        for offset in range(0, 500, 100):  # First 5 pages
            count = parse_science_direct(query, offset)
            if count < 20: break
        
        # ACM
        #for page in range(1, 16):  # First 5 pages
            #count = parse_acm(query, page)
            #if count < 10: break
    
    driver.quit()

if __name__ == '__main__':
    main()

Processing: Benchmark+Dataset+Program+Repair+LLM
No ScienceDirect results found
Processing: Benchmark+Dataset+Requirement+Generation+LLM
No ScienceDirect results found


In [ ]:
'Benchmark+Dataset+Code+Generation+LLM+Large+Language+Model',
'Benchmark+Dataset+Code+Summarization+LLM',
'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
'Benchmark+Dataset+Patch+Generation+LLM',
'Benchmark+Dataset+Code+Optimization+LLM',
'Benchmark+Dataset+Code+Translation+LLM',
'Benchmark+Dataset+Program+Repair+LLM',
'Benchmark+Dataset+Requirement+Generation+LLM',
'Benchmark+Dataset+Software+Development+LLM',
'Benchmark+Dataset+Software+Engineering+LLM',
'Benchmark+Dataset+Code+Review+LLM',



    


In [ ]:
'Benchmark+Code+Generation',
'Benchmark+Code+Summarization',
'Benchmark+Code+Test+Cases+Generation',
'Benchmark+Patch+Generation',
'Benchmark+Code+Optimization',
'Benchmark+Code+Translation',
'Benchmark+Code+Test+Cases+Generation',
'Benchmark+Code+Test+Cases+Generation',

In [ ]:
import bs4
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import csv
import re
import json
from selenium import webdriver
import random
from datetime import datetime
from urllib.parse import quote

driver = webdriver.Chrome()

def loadtxt():
    return [line.strip() for line in open("query.txt", "r")]

def jsonWriter(data, filename):
    with open(f'{filename}.json', 'a') as f:
        json.dump(data, f, ensure_ascii=False)

def newWriter(row, filename):
    with open(f'{filename}.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)

def randomize_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    ]
    return {'User-Agent': random.choice(user_agents)}

def handle_cookie_consent():
    try:
        cookie_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept') or contains(., 'Agree') or contains(., 'OK')]"))
        )
        cookie_btn.click()
        time.sleep(2)
    except TimeoutException:
        pass

def is_recent_publication(year_text):
    try:
        year = int(re.search(r'\d{4}', year_text).group())
        return year >= 2020
    except (AttributeError, ValueError):
        return False
        
def google_scholar_scraper(query, start=0):
    base_url = f"https://scholar.google.com/scholar?start={start}&q={query}&hl=en&as_sdt=0,5&as_ylo=2020"
    driver.get(base_url)
    if start == 0:
        time.sleep(20)
    else:
        time.sleep(random.uniform(2,4))
    
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "gs_ri"))
        )
        
        papers = []
        for result in results:
            try:
                title = result.find_element(By.TAG_NAME, 'h3').text
                link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
                year_text = result.find_element(By.CLASS_NAME, 'gs_a').text
                
                if is_recent_publication(year_text):
                    papers.append([title, link, year_text])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'GoogleScholarPapers')
        
        return len(papers)

    except TimeoutException:
        print("No results found or page didn't load")
        return 0

def parse_ieee(keyword, page=0):
    url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={keyword}&pageNumber={page}&ranges=2020_{datetime.now().year}_Year"
    driver.get(url)
    time.sleep(3)
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".List-results-items"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, "h3 a").text
                year_text = item.find_element(By.XPATH, ".//*[contains(text(), 'Year:')]").text
                
                if is_recent_publication(year_text):
                    papers.append([title, f"IEEE {year_text}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'IEEEPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No IEEE results found")
        return 0

def parse_science_direct(keyword, offset=0):
    """Scrapes ScienceDirect search results for paper titles and links."""
    
    url = f'https://www.sciencedirect.com/search?date=2020-2025&tak={keyword}&offset={offset}'
    
    driver.get(url)
    time.sleep(3)
    handle_cookie_consent()

    papers = []
    
    try:
        # Wait until search results load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "result-item-container"))
        )
        
        results = driver.find_elements(By.CLASS_NAME, "result-item-container")

        for item in results:
            try:
                title_element = item.find_element(By.TAG_NAME, 'h2').find_element(By.TAG_NAME, 'a')
                title = title_element.text
                link = title_element.get_attribute('href')
                papers.append([title, link])
            except Exception as e:
                print(f"Skipping item due to error: {e}")
                continue
        
        for paper in papers:
            newWriter(paper, 'ScienceDirectPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No ScienceDirect results found")
        return 0

def parse_acm(query, page):
    #url = f"https://dl.acm.org/action/doSearch?field1=TitleAbstract&text1={query}&AfterYear=2020&startPage={page-1}&pageSize=50"
    #url = f"https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField={query}&pageSize=20&startPage={page-1}"
    url = f'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=Title&text1={query}&field2=Abstract&text2={query}&AfterYear=2020&pageSize=20&startPage={page-1}'
    #'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField={query}&pageSize=20&startPage={page-1}'
    #https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField=+AND+AllField%3A%28benchmark+dataset+code+generation+llm%29&pageSize=20&startPage=1
    response = requests.get(url, headers=randomize_user_agent())
    soup = BeautifulSoup(response.text, 'html.parser')
    
    papers = []
    for item in soup.select('.issue-item__title'):
        try:
            title = item.text.strip()
            link = "https://dl.acm.org" + item.find('a')['href']
            year_text = item.find_next(class_='bookPubDate').text
            
            if is_recent_publication(year_text):
                papers.append([title, link, year_text])
        except:
            continue
    
    for paper in papers:
        newWriter(paper, 'ACMPapers')
    
    print(f'ACM: {len(papers)}')
    return len(papers)

def main():
    queries = [
        'Benchmark+Dataset+Code+Generation+LLM',
        'Benchmark+Dataset+Code+Summarization+LLM',
        'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
        'Benchmark+Dataset+Patch+Generation+LLM',
        'Benchmark+Dataset+Code+Optimization+LLM',
        'Benchmark+Dataset+Code+Translation+LLM',
        'Benchmark+Dataset+Program+Repair+LLM',
        'Benchmark+Dataset+Requirement+Generation+LLM',
        'Benchmark+Dataset+Software+Development+LLM',
        'Benchmark+Dataset+Software+Engineering+LLM',
        'Benchmark+Dataset+Code+Review+LLM',
        'Benchmark+Dataset+Code+Generation+Large+Language+Model',
        'Benchmark+Dataset+Code+Summarization+Large+Language+Model',
        'Benchmark+Dataset+Code+Test+Cases+Generation+Large+Language+Model',
        'Benchmark+Dataset+Patch+Generation+Large+Language+Model',
        'Benchmark+Dataset+Code+Optimization+Large+Language+Model',
        'Benchmark+Dataset+Code+Translation+Large+Language+Model',
        'Benchmark+Dataset+Program+Repair+Large+Language+Model',
        'Benchmark+Dataset+Requirement+Generation+Large+Language+Model',
        'Benchmark+Dataset+Software+Development+Large+Language+Model',
        'Benchmark+Dataset+Software+Engineering+Large+Language+Model',
        'Benchmark+Dataset+Code+Review+Large+Language+Model'
        ]
    
    for query in queries:
        print(f"Processing: {query}")
        
        # Google Scholar
        count_total = 0
        for page in range(0, 20):  # First 5 pages
            count = google_scholar_scraper(query, start=page*10)
            count_total+=count
            if count == 0: 
                print(count_total)
                print(f'number of papers for google scholar: {count_total}')
                break
            if page == 19:
                print(f'number of papers for google scholar: {count_total}')
        
        # IEEE
        #count_total = 0
        #for page in range(0, 20):  # First 5 pages
            #count = parse_ieee(query, page)
            #count_total+=count
            #if count == 0: 
                #print(f'number of papers for IEEE: {count_total}')
                #break

        # ScienceDirect
        #count_total = 0
        #for offset in range(0, 250, 25):  # First 5 pages
            #count = parse_science_direct(query, offset)
            #count_total+=count
            #if count == 0:  
                #print(f'number of papers for science direct: {count_total}') 
                #break
        
        # ACM
        count_total = 0
        for page in range(1, 6):  # First 5 pages
            count = parse_acm(query, page)
            count_total+=count
            if count == 0: 
                print(count_total)
                print(f'number of papers for google scholar: {count_total}')
                break
            if page == 11:
                print(f'number of papers for google scholar: {count_total}')
    
    driver.quit()

if __name__ == '__main__':
    main()

Processing: Benchmark+Dataset+Code+Generation+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 69
No ScienceDirect results found
number of papers for science direct: 10
Processing: Benchmark+Dataset+Code+Summarization+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 6
No ScienceDirect results found
number of papers for science direct: 1
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 24
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Patch+Generation+LLM
number of papers for google scholar: 199
No IEEE results found
number of papers for IEEE: 2
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Code+Optimization+LLM
No results found or page didn't load
179
number of papers for google scholar: 179


In [ ]:
import bs4
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import csv
import re
import json
from selenium import webdriver
import random
from datetime import datetime
from urllib.parse import quote

driver = webdriver.Chrome()

def loadtxt():
    return [line.strip() for line in open("query.txt", "r")]

def jsonWriter(data, filename):
    with open(f'{filename}.json', 'a') as f:
        json.dump(data, f, ensure_ascii=False)

def newWriter(row, filename):
    with open(f'{filename}.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)

def randomize_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    ]
    return {'User-Agent': random.choice(user_agents)}

def handle_cookie_consent():
    try:
        cookie_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept') or contains(., 'Agree') or contains(., 'OK')]"))
        )
        cookie_btn.click()
        time.sleep(2)
    except TimeoutException:
        pass

def is_recent_publication(year_text):
    try:
        year = int(re.search(r'\d{4}', year_text).group())
        return year >= 2020
    except (AttributeError, ValueError):
        return False
        
def google_scholar_scraper(query, start=0):
    base_url = f"https://scholar.google.com/scholar?start={start}&q={query}&hl=en&as_sdt=0,5&as_ylo=2020"
    driver.get(base_url)
    if start == 0:
        time.sleep(30)
    else:
        time.sleep(random.uniform(2,4))
    
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "gs_ri"))
        )
        
        papers = []
        for result in results:
            try:
                title = result.find_element(By.TAG_NAME, 'h3').text
                link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
                year_text = result.find_element(By.CLASS_NAME, 'gs_a').text
                
                if is_recent_publication(year_text):
                    papers.append([title, link, year_text])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'GoogleScholarPapers')
        
        return len(papers)

    except TimeoutException:
        print("No results found or page didn't load")
        return 0

def parse_ieee(keyword, page=0):
    url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={keyword}&pageNumber={page}&ranges=2020_{datetime.now().year}_Year"
    driver.get(url)
    time.sleep(3)
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".List-results-items"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, "h3 a").text
                year_text = item.find_element(By.XPATH, ".//*[contains(text(), 'Year:')]").text
                
                if is_recent_publication(year_text):
                    papers.append([title, f"IEEE {year_text}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'IEEEPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No IEEE results found")
        return 0

def parse_science_direct(keyword, offset=0):
    """Scrapes ScienceDirect search results for paper titles and links."""
    
    url = f'https://www.sciencedirect.com/search?date=2020-2025&tak={keyword}&offset={offset}'
    
    driver.get(url)
    time.sleep(3)
    handle_cookie_consent()

    papers = []
    
    try:
        # Wait until search results load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "result-item-container"))
        )
        
        results = driver.find_elements(By.CLASS_NAME, "result-item-container")

        for item in results:
            try:
                title_element = item.find_element(By.TAG_NAME, 'h2').find_element(By.TAG_NAME, 'a')
                title = title_element.text
                link = title_element.get_attribute('href')
                papers.append([title, link])
            except Exception as e:
                print(f"Skipping item due to error: {e}")
                continue
        
        for paper in papers:
            newWriter(paper, 'ScienceDirectPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No ScienceDirect results found")
        return 0

def parse_acm(query, page):
    #url = f"https://dl.acm.org/action/doSearch?field1=TitleAbstract&text1={query}&AfterYear=2020&startPage={page-1}&pageSize=50"
    url = f"https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&ABSTRACT={query}&pageSize=20&startPage={page-1}"
    #url = f"https://dl.acm.org/action/doSearch?field1=TitleAbstract&text1={query}&AfterYear=2020&startPage={page-1}&pageSize=20"#f'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=Title&text1={query}&field2=Abstract&text2={query}&AfterYear=2020&pageSize=20&startPage={page-1}'
    #'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField={query}&pageSize=20&startPage={page-1}'
    #https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField=+AND+AllField%3A%28benchmark+dataset+code+generation+llm%29&pageSize=20&startPage=1
    response = requests.get(url, headers=randomize_user_agent())
    soup = BeautifulSoup(response.text, 'html.parser')
    
    papers = []
    for item in soup.select('.issue-item__title'):
        try:
            title = item.text.strip()
            link = "https://dl.acm.org" + item.find('a')['href']
            year_text = item.find_next(class_='bookPubDate').text
            
            if is_recent_publication(year_text):
                papers.append([title, link, year_text])
        except:
            continue
    
    for paper in papers:
        newWriter(paper, 'ACMPapers')
    
    print(f'ACM: {len(papers)}')
    return len(papers)

def main():
    queries = [
        'Benchmark+Dataset+Code+Generation+LLM',
        'Benchmark+Dataset+Code+Summarization+LLM',
        'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
        'Benchmark+Dataset+Patch+Generation+LLM',
        'Benchmark+Dataset+Code+Optimization+LLM',
        'Benchmark+Dataset+Code+Translation+LLM',
        'Benchmark+Dataset+Program+Repair+LLM',
        'Benchmark+Dataset+Requirement+Generation+LLM',
        'Benchmark+Dataset+Software+Development+LLM',
        'Benchmark+Dataset+Software+Engineering+LLM',
        'Benchmark+Dataset+Code+Review+LLM',
        'Benchmark+Dataset+Code+Generation+Large+Language+Model',
        'Benchmark+Dataset+Code+Summarization+Large+Language+Model',
        'Benchmark+Dataset+Code+Test+Cases+Generation+Large+Language+Model',
        'Benchmark+Dataset+Patch+Generation+Large+Language+Model',
        'Benchmark+Dataset+Code+Optimization+Large+Language+Model',
        'Benchmark+Dataset+Code+Translation+Large+Language+Model',
        'Benchmark+Dataset+Program+Repair+Large+Language+Model',
        'Benchmark+Dataset+Requirement+Generation+Large+Language+Model',
        'Benchmark+Dataset+Software+Development+Large+Language+Model',
        'Benchmark+Dataset+Software+Engineering+Large+Language+Model',
        'Benchmark+Dataset+Code+Review+Large+Language+Model'
        ]
    
    for query in queries:
        print(f"Processing: {query}")
        
        # Google Scholar
        count_total = 0
        for page in range(0, 6):  # First 5 pages
            count = google_scholar_scraper(query, start=page*10)
            count_total+=count
            if count == 0: 
                print(count_total)
                print(f'number of papers for google scholar: {count_total}')
                break
            if page == 5:
                print(f'number of papers for google scholar: {count_total}')
        
        # IEEE
        count_total = 0
        for page in range(0, 20):  # First 5 pages
            count = parse_ieee(query, page)
            count_total+=count
            if count == 0: 
                print(f'number of papers for IEEE: {count_total}')
                break

        # ScienceDirect
        count_total = 0
        for offset in range(0, 250, 25):  # First 5 pages
            count = parse_science_direct(query, offset)
            count_total+=count
            if count == 0:  
                print(f'number of papers for science direct: {count_total}') 
                break
        
        # ACM
        count_total = 0
        for page in range(1, 6):  # First 5 pages
            count = parse_acm(query, page)
            count_total+=count
            if count == 0: 
                print(count_total)
                print(f'number of papers for google scholar: {count_total}')
                break
            if page == 5:
                print(f'number of papers for google scholar: {count_total}')
    
    driver.quit()

if __name__ == '__main__':
    main()

Processing: Benchmark+Dataset+Code+Generation+LLM
ACM: 19
ACM: 11
ACM: 0
30
number of papers for google scholar: 30
Processing: Benchmark+Dataset+Code+Summarization+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
ACM: 2
ACM: 0
2
number of papers for google scholar: 2
Processing: Benchmark+Dataset+Patch+Generation+LLM
ACM: 5
ACM: 0
5
number of papers for google scholar: 5
Processing: Benchmark+Dataset+Code+Optimization+LLM
ACM: 4
ACM: 0
4
number of papers for google scholar: 4
Processing: Benchmark+Dataset+Code+Translation+LLM
ACM: 7
ACM: 0
7
number of papers for google scholar: 7
Processing: Benchmark+Dataset+Program+Repair+LLM
ACM: 8
ACM: 0
8
number of papers for google scholar: 8
Processing: Benchmark+Dataset+Requirement+Generation+LLM
ACM: 19
ACM: 4
ACM: 0
23
number of papers for google scholar: 23
Processing: Benchmark+Dataset+Software+Development+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchma

In [ ]:
import bs4
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import csv
import re
import json
from selenium import webdriver
import random
from datetime import datetime
from urllib.parse import quote

driver = webdriver.Chrome()

def loadtxt():
    return [line.strip() for line in open("query.txt", "r")]

def jsonWriter(data, filename):
    with open(f'{filename}.json', 'a') as f:
        json.dump(data, f, ensure_ascii=False)

def newWriter(row, filename):
    with open(f'{filename}.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)

def randomize_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    ]
    return {'User-Agent': random.choice(user_agents)}

def handle_cookie_consent():
    try:
        cookie_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept') or contains(., 'Agree') or contains(., 'OK')]"))
        )
        cookie_btn.click()
        time.sleep(2)
    except TimeoutException:
        pass

def is_recent_publication(year_text):
    try:
        year = int(re.search(r'\d{4}', year_text).group())
        return year >= 2020
    except (AttributeError, ValueError):
        return False
        
def google_scholar_scraper(query, start=0):
    base_url = f"https://scholar.google.com/scholar?start={start}&q={query}&hl=en&as_sdt=0,5&as_ylo=2020"
    driver.get(base_url)
    if start == 0:
        time.sleep(30)
    else:
        time.sleep(random.uniform(2,4))
    
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "gs_ri"))
        )
        
        papers = []
        for result in results:
            try:
                title = result.find_element(By.TAG_NAME, 'h3').text
                link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
                year_text = result.find_element(By.CLASS_NAME, 'gs_a').text
                
                if is_recent_publication(year_text):
                    papers.append([title, link, year_text])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'GoogleScholarPapers')
        
        return len(papers)

    except TimeoutException:
        print("No results found or page didn't load")
        return 0

def parse_ieee(keyword, page=0):
    url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?queryText={keyword}&pageNumber={page}&ranges=2020_{datetime.now().year}_Year"
    driver.get(url)
    time.sleep(3)
    
    papers = []
    try:
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".List-results-items"))
        )
        
        for item in results:
            try:
                title = item.find_element(By.CSS_SELECTOR, "h3 a").text
                year_text = item.find_element(By.XPATH, ".//*[contains(text(), 'Year:')]").text
                
                if is_recent_publication(year_text):
                    papers.append([title, f"IEEE {year_text}"])
            except:
                continue
        
        for paper in papers:
            newWriter(paper, 'IEEEPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No IEEE results found")
        return 0

def parse_science_direct(keyword, offset=0):
    """Scrapes ScienceDirect search results for paper titles and links."""
    
    url = f'https://www.sciencedirect.com/search?date=2020-2025&tak={keyword}&offset={offset}'
    
    driver.get(url)
    time.sleep(3)
    handle_cookie_consent()

    papers = []
    
    try:
        # Wait until search results load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "result-item-container"))
        )
        
        results = driver.find_elements(By.CLASS_NAME, "result-item-container")

        for item in results:
            try:
                title_element = item.find_element(By.TAG_NAME, 'h2').find_element(By.TAG_NAME, 'a')
                title = title_element.text
                link = title_element.get_attribute('href')
                papers.append([title, link])
            except Exception as e:
                print(f"Skipping item due to error: {e}")
                continue
        
        for paper in papers:
            newWriter(paper, 'ScienceDirectPapers')
        
        return len(papers)
    
    except TimeoutException:
        print("No ScienceDirect results found")
        return 0


def parse_acm(query, page):
    #url = f"https://dl.acm.org/action/doSearch?field1=TitleAbstract&text1={query}&AfterYear=2020&startPage={page-1}&pageSize=50"
    url = f"https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&TITLE={query}&pageSize=20&startPage={page-1}"
    #url = f"https://dl.acm.org/action/doSearch?field1=TitleAbstract&text1={query}&AfterYear=2020&startPage={page-1}&pageSize=20"#f'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=Title&text1={query}&field2=Abstract&text2={query}&AfterYear=2020&pageSize=20&startPage={page-1}'
    #'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField={query}&pageSize=20&startPage={page-1}'
    #https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&AfterYear=2020&AllField=+AND+AllField%3A%28benchmark+dataset+code+generation+llm%29&pageSize=20&startPage=1
    response = requests.get(url, headers=randomize_user_agent())
    soup = BeautifulSoup(response.text, 'html.parser')
    
    papers = []
    for item in soup.select('.issue-item__title'):
        try:
            title = item.text.strip()
            link = "https://dl.acm.org" + item.find('a')['href']
            year_text = item.find_next(class_='bookPubDate').text
            
            if is_recent_publication(year_text):
                papers.append([title, link, year_text])
        except:
            continue
    
    for paper in papers:
        newWriter(paper, 'ACMPapers')
    
    print(f'ACM: {len(papers)}')
    return len(papers)

def main():
    queries = [
        'Benchmark+Dataset+Code+Generation+LLM',
        'Benchmark+Dataset+Code+Summarization+LLM',
        'Benchmark+Dataset+Code+Test+Cases+Generation+LLM',
        'Benchmark+Dataset+Patch+Generation+LLM',
        'Benchmark+Dataset+Code+Optimization+LLM',
        'Benchmark+Dataset+Code+Translation+LLM',
        'Benchmark+Dataset+Program+Repair+LLM',
        'Benchmark+Dataset+Requirement+Generation+LLM',
        'Benchmark+Dataset+Software+Development+LLM',
        'Benchmark+Dataset+Software+Engineering+LLM',
        'Benchmark+Dataset+Code+Review+LLM',
        'Benchmark+Dataset+Code+Generation+Large+Language+Model',
        'Benchmark+Dataset+Code+Summarization+Large+Language+Model',
        'Benchmark+Dataset+Code+Test+Cases+Generation+Large+Language+Model',
        'Benchmark+Dataset+Patch+Generation+Large+Language+Model',
        'Benchmark+Dataset+Code+Optimization+Large+Language+Model',
        'Benchmark+Dataset+Code+Translation+Large+Language+Model',
        'Benchmark+Dataset+Program+Repair+Large+Language+Model',
        'Benchmark+Dataset+Requirement+Generation+Large+Language+Model',
        'Benchmark+Dataset+Software+Development+Large+Language+Model',
        'Benchmark+Dataset+Software+Engineering+Large+Language+Model',
        'Benchmark+Dataset+Code+Review+Large+Language+Model'
        ]
    
    for query in queries:
        print(f"Processing: {query}")
        
        # Google Scholar
        count_total = 0
        for page in range(0, 6):  # First 5 pages
            count = google_scholar_scraper(query, start=page*10)
            count_total+=count
            if count == 0: 
                print(count_total)
                print(f'number of papers for google scholar: {count_total}')
                break
            if page == 5:
                print(f'number of papers for google scholar: {count_total}')
        
        # IEEE
        count_total = 0
        for page in range(0, 20):  # First 5 pages
            count = parse_ieee(query, page)
            count_total+=count
            if count == 0: 
                print(f'number of papers for IEEE: {count_total}')
                break

        # ScienceDirect
        count_total = 0
        for offset in range(0, 250, 25):  # First 5 pages
            count = parse_science_direct(query, offset)
            count_total+=count
            if count == 0:  
                print(f'number of papers for science direct: {count_total}') 
                break
        
        # ACM
        count_total = 0
        for page in range(1, 6):  # First 5 pages
            count = parse_acm(query, page)
            count_total+=count
            if count == 0: 
                print(count_total)
                print(f'number of papers for google scholar: {count_total}')
                break
            if page == 5:
                print(f'number of papers for google scholar: {count_total}')
    
    driver.quit()

if __name__ == '__main__':
    main()

Processing: Benchmark+Dataset+Code+Generation+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Summarization+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Patch+Generation+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Optimization+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Translation+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Program+Repair+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Requirement+Generation+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Software+Development+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Software+Engineering+LLM
ACM: 0
0
number of papers for goog

In [27]:
logs = """Processing: Benchmark+Dataset+Code+Generation+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 69
No ScienceDirect results found
number of papers for science direct: 10
Processing: Benchmark+Dataset+Code+Summarization+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 6
No ScienceDirect results found
number of papers for science direct: 1
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 24
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Patch+Generation+LLM
number of papers for google scholar: 199
No IEEE results found
number of papers for IEEE: 2
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Code+Optimization+LLM
No results found or page didn't load
179
number of papers for google scholar: 179
No IEEE results found
number of papers for IEEE: 8
No ScienceDirect results found
number of papers for science direct: 4
Processing: Benchmark+Dataset+Code+Translation+LLM
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 18
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Program+Repair+LLM
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 4
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Requirement+Generation+LLM
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 8
No ScienceDirect results found
number of papers for science direct: 3
Processing: Benchmark+Dataset+Software+Development+LLM
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 40
No ScienceDirect results found
number of papers for science direct: 2
Processing: Benchmark+Dataset+Software+Engineering+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 56
No ScienceDirect results found
number of papers for science direct: 1
Processing: Benchmark+Dataset+Code+Review+LLM
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 8
No ScienceDirect results found
number of papers for science direct: 2
Processing: Benchmark+Dataset+Code+Generation+Large+Language+Model
number of papers for google scholar: 198
No IEEE results found
number of papers for IEEE: 141
No ScienceDirect results found
number of papers for science direct: 17
Processing: Benchmark+Dataset+Code+Summarization+Large+Language+Model
number of papers for google scholar: 197
No IEEE results found
number of papers for IEEE: 28
No ScienceDirect results found
number of papers for science direct: 2
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+Large+Language+Model
number of papers for google scholar: 200
No IEEE results found
number of papers for IEEE: 38
No ScienceDirect results found
number of papers for science direct: 1
Processing: Benchmark+Dataset+Patch+Generation+Large+Language+Model
number of papers for google scholar: 197
No IEEE results found
number of papers for IEEE: 14
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Code+Optimization+Large+Language+Model
number of papers for google scholar: 198
No IEEE results found
number of papers for IEEE: 34
No ScienceDirect results found
number of papers for science direct: 8
Processing: Benchmark+Dataset+Code+Translation+Large+Language+Model
number of papers for google scholar: 199
No IEEE results found
number of papers for IEEE: 44
No ScienceDirect results found
number of papers for science direct: 1
Processing: Benchmark+Dataset+Program+Repair+Large+Language+Model
number of papers for google scholar: 199
No IEEE results found
number of papers for IEEE: 18
No ScienceDirect results found
number of papers for science direct: 0
Processing: Benchmark+Dataset+Requirement+Generation+Large+Language+Model
number of papers for google scholar: 199
No IEEE results found
number of papers for IEEE: 24
No ScienceDirect results found
number of papers for science direct: 4
Processing: Benchmark+Dataset+Software+Development+Large+Language+Model
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 83
No ScienceDirect results found
number of papers for science direct: 3
Processing: Benchmark+Dataset+Software+Engineering+Large+Language+Model
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 145
No ScienceDirect results found
number of papers for science direct: 2
Processing: Benchmark+Dataset+Code+Review+Large+Language+Model
No results found or page didn't load
0
number of papers for google scholar: 0
No IEEE results found
number of papers for IEEE: 36
No ScienceDirect results found
number of papers for science direct: 3

Processing: Benchmark+Dataset+Code+Generation+LLM
ACM: 19
ACM: 11
ACM: 0
30
number of papers for google scholar: 30
Processing: Benchmark+Dataset+Code+Summarization+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+LLM
ACM: 2
ACM: 0
2
number of papers for google scholar: 2
Processing: Benchmark+Dataset+Patch+Generation+LLM
ACM: 5
ACM: 0
5
number of papers for google scholar: 5
Processing: Benchmark+Dataset+Code+Optimization+LLM
ACM: 4
ACM: 0
4
number of papers for google scholar: 4
Processing: Benchmark+Dataset+Code+Translation+LLM
ACM: 7
ACM: 0
7
number of papers for google scholar: 7
Processing: Benchmark+Dataset+Program+Repair+LLM
ACM: 8
ACM: 0
8
number of papers for google scholar: 8
Processing: Benchmark+Dataset+Requirement+Generation+LLM
ACM: 19
ACM: 4
ACM: 0
23
number of papers for google scholar: 23
Processing: Benchmark+Dataset+Software+Development+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Software+Engineering+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Review+LLM
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Generation+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Summarization+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Test+Cases+Generation+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Patch+Generation+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Optimization+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Translation+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Program+Repair+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Requirement+Generation+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Software+Development+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Software+Engineering+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0
Processing: Benchmark+Dataset+Code+Review+Large+Language+Model
ACM: 0
0
number of papers for google scholar: 0


"""

In [30]:
import re
from collections import defaultdict

def parse_logs(logs):
    query_stats = defaultdict(lambda: {"Google Scholar": 0, "IEEE": 0, "ACM": 0, "ScienceDirect": 0})
    current_query = None
    
    for line in logs.split("\n"):
        line = line.strip()
        
        # Detect new query
        if line.startswith("Processing:"):
            current_query = line.replace("Processing: ", "")
        
        elif "number of papers for google scholar:" in line:
            query_stats[current_query]["Google Scholar"] = 60        
        elif "number of papers for IEEE:" in line:
            query_stats[current_query]["IEEE"] = int(re.search(r'\d+', line).group())
        
        elif "number of papers for science direct:" in line:
            query_stats[current_query]["ScienceDirect"] = int(re.search(r'\d+', line).group())
        
        elif line.startswith("ACM:"):
            count = int(re.search(r'\d+', line).group())
            query_stats[current_query]["ACM"] += count
    
    return query_stats

def display_stats(query_stats):
    print(f"{'Query':<60}{'Google Scholar':<15}{'IEEE':<10}{'ACM':<10}{'ScienceDirect':<15}")
    print("=" * 110)
    for query, counts in query_stats.items():
        print(f"{query:<60}{counts['Google Scholar']:<15}{counts['IEEE']:<10}{counts['ACM']:<10}{counts['ScienceDirect']:<15}")

# Example usage
query_stats = parse_logs(logs)
display_stats(query_stats)


Query                                                       Google Scholar IEEE      ACM       ScienceDirect  
Benchmark+Dataset+Code+Generation+LLM                       60             69        30        10             
Benchmark+Dataset+Code+Summarization+LLM                    60             6         0         1              
Benchmark+Dataset+Code+Test+Cases+Generation+LLM            60             24        2         0              
Benchmark+Dataset+Patch+Generation+LLM                      60             2         5         0              
Benchmark+Dataset+Code+Optimization+LLM                     60             8         4         4              
Benchmark+Dataset+Code+Translation+LLM                      60             18        7         0              
Benchmark+Dataset+Program+Repair+LLM                        60             4         8         0              
Benchmark+Dataset+Requirement+Generation+LLM                60             8         23        3              
B

In [31]:
import pandas as pd
# Convert to DataFrame
df = pd.DataFrame.from_dict(query_stats, orient='index').reset_index()
df.rename(columns={'index': 'Query'}, inplace=True)

df

,Query,Google Scholar,IEEE,ACM,ScienceDirect
0,Benchmark+Dataset+Code+Generation+LLM,60,69,30,10
1,Benchmark+Dataset+Code+Summarization+LLM,60,6,0,1
2,Benchmark+Dataset+Code+Test+Cases+Generation+LLM,60,24,2,0
3,Benchmark+Dataset+Patch+Generation+LLM,60,2,5,0
4,Benchmark+Dataset+Code+Optimization+LLM,60,8,4,4
5,Benchmark+Dataset+Code+Translation+LLM,60,18,7,0
6,Benchmark+Dataset+Program+Repair+LLM,60,4,8,0
7,Benchmark+Dataset+Requirement+Generation+LLM,60,8,23,3
8,Benchmark+Dataset+Software+Development+LLM,60,40,0,2
9,Benchmark+Dataset+Software+Engineering+LLM,60,56,0,1


In [ ]:
# Save to CSV


In [35]:
# Add a final row with sums of all columns except 'Query'
sum_row = df.drop(columns=['Query']).sum().to_dict()
sum_row['Query'] = 'Total'
df = pd.concat([df, pd.DataFrame([sum_row])], ignore_index=True)
df.to_csv("overall_stats.csv", index=False)

In [36]:
df

,Query,Google Scholar,IEEE,ACM,ScienceDirect
0,Benchmark+Dataset+Code+Generation+LLM,60,69,30,10
1,Benchmark+Dataset+Code+Summarization+LLM,60,6,0,1
2,Benchmark+Dataset+Code+Test+Cases+Generation+LLM,60,24,2,0
3,Benchmark+Dataset+Patch+Generation+LLM,60,2,5,0
4,Benchmark+Dataset+Code+Optimization+LLM,60,8,4,4
5,Benchmark+Dataset+Code+Translation+LLM,60,18,7,0
6,Benchmark+Dataset+Program+Repair+LLM,60,4,8,0
7,Benchmark+Dataset+Requirement+Generation+LLM,60,8,23,3
8,Benchmark+Dataset+Software+Development+LLM,60,40,0,2
9,Benchmark+Dataset+Software+Engineering+LLM,60,56,0,1


In [7]:
import pandas as pd 

t = pd.read_csv('GoogleScholarPapers.csv', names=['Title', 'Link', 'Year'])
t

,Title,Link,Year
0,Test-Driven Development and LLM-based Code Gen...,https://dl.acm.org/doi/abs/10.1145/3691620.369...,"NS Mathews, M Nagappan - Proceedings of the 39..."
1,[PDF] Beyond synthetic benchmarks: Assessing r...,https://www.researchgate.net/profile/Amirkia-R...,"AR Oskooei, MS Babacan, E Yağcı… - The 14th In..."
2,Classeval: A manually-crafted benchmark for ev...,https://arxiv.org/abs/2308.01861,"X Du, M Liu, K Wang, H Wang, J Liu, Y Chen… - ..."
3,A Survey on Large Language Models for Code Gen...,https://arxiv.org/abs/2406.00515,"J Jiang, F Wang, J Shen, S Kim, S Kim - arXiv ..."
4,When llm-based code generation meets the softw...,https://arxiv.org/abs/2403.15852,"F Lin, DJ Kim - arXiv preprint arXiv:2403.1585..."
...,...,...,...
1333,Gpt4graph: Can large language models understan...,https://arxiv.org/abs/2305.15066,"J Guo, L Du, H Liu, M Zhou, X He, S Han - arXi..."
1334,Benchmarking and defending against indirect pr...,https://arxiv.org/abs/2312.14197,"J Yi, Y Xie, B Zhu, E Kiciman, G Sun, X Xie… -..."
1335,mplug-paperowl: Scientific diagram analysis wi...,https://dl.acm.org/doi/abs/10.1145/3664647.368...,"A Hu, Y Shi, H Xu, J Ye, Q Ye, M Yan, C Li… - ..."
1336,Genegpt: Augmenting large language models with...,https://academic.oup.com/bioinformatics/articl...,"Q Jin, Y Yang, Q Chen, Z Lu - Bioinformatics, ..."


In [8]:
t.drop_duplicates(subset=['Title'], keep='first')

,Title,Link,Year
0,Test-Driven Development and LLM-based Code Gen...,https://dl.acm.org/doi/abs/10.1145/3691620.369...,"NS Mathews, M Nagappan - Proceedings of the 39..."
1,[PDF] Beyond synthetic benchmarks: Assessing r...,https://www.researchgate.net/profile/Amirkia-R...,"AR Oskooei, MS Babacan, E Yağcı… - The 14th In..."
2,Classeval: A manually-crafted benchmark for ev...,https://arxiv.org/abs/2308.01861,"X Du, M Liu, K Wang, H Wang, J Liu, Y Chen… - ..."
3,A Survey on Large Language Models for Code Gen...,https://arxiv.org/abs/2406.00515,"J Jiang, F Wang, J Shen, S Kim, S Kim - arXiv ..."
4,When llm-based code generation meets the softw...,https://arxiv.org/abs/2403.15852,"F Lin, DJ Kim - arXiv preprint arXiv:2403.1585..."
...,...,...,...
1303,"Benchmark evaluations, applications, and chall...",https://arxiv.org/abs/2501.02189,"Z Li, X Wu, H Du, H Nghiem, G Shi - arXiv prep..."
1324,Large language models for code analysis: Do {L...,https://www.usenix.org/conference/usenixsecuri...,"C Fang, N Miao, S Srivastav, J Liu, R Zhang… -..."
1331,ROBBIE: Robust bias evaluation of large genera...,https://arxiv.org/abs/2311.18140,"D Esiobu, X Tan, S Hosseini, M Ung, Y Zhang… -..."
1335,mplug-paperowl: Scientific diagram analysis wi...,https://dl.acm.org/doi/abs/10.1145/3664647.368...,"A Hu, Y Shi, H Xu, J Ye, Q Ye, M Yan, C Li… - ..."


In [16]:
t = pd.read_csv('~/Downloads/survey inclusion exclusion criteria - include_exclude.csv')
t.drop_duplicates(subset=['title'], keep='first', inplace=True)
t.to_csv('survey inclusion exclusion criteria - include_exclude.csv', index=False)

In [ ]:
import pandas as pd 

t = pd.read_csv('survey inclusion exclusion - include_exclude no duplicates.csv')


,title,link,source,venue,include/exclude,questions,future use
0,Test-Driven Development and LLM-based Code Gen...,https://dl.acm.org/doi/abs/10.1145/3691620.369...,Google Scholar,NaN,exclude,NaN,NaN
1,[PDF] Beyond synthetic benchmarks: Assessing r...,https://www.researchgate.net/profile/Amirkia-R...,Google Scholar,wcse,?,NaN,NaN
2,Classeval: A manually-crafted benchmark for ev...,https://arxiv.org/abs/2308.01861,Google Scholar,arvix,include,NaN,NaN
3,A Survey on Large Language Models for Code Gen...,https://arxiv.org/abs/2406.00515,Google Scholar,NaN,exclude,NaN,NaN
4,When llm-based code generation meets the softw...,https://arxiv.org/abs/2403.15852,Google Scholar,NaN,exclude,NaN,NaN
...,...,...,...,...,...,...,...
494,"Benchmark evaluations, applications, and chall...",https://arxiv.org/abs/2501.02189,Google Scholar,NaN,NaN,NaN,NaN
495,Large language models for code analysis: Do {L...,https://www.usenix.org/conference/usenixsecuri...,Google Scholar,NaN,NaN,NaN,NaN
496,ROBBIE: Robust bias evaluation of large genera...,https://arxiv.org/abs/2311.18140,Google Scholar,NaN,NaN,NaN,NaN
497,mplug-paperowl: Scientific diagram analysis wi...,https://dl.acm.org/doi/abs/10.1145/3664647.368...,Google Scholar,NaN,NaN,NaN,NaN


In [20]:
ieee = pd.read_csv('IEEEPapers.csv', names=['title', 'year'])
ieee['source'] = 'IEEE'
ieee

,title,year,source
0,RTL-Repo: A Benchmark for Evaluating LLMs on L...,IEEE Year: 2024,IEEE
1,FormalEval: A Method for Automatic Evaluation ...,IEEE Year: 2024,IEEE
2,Test-Driven Development and LLM-based Code Gen...,IEEE Year: 2024,IEEE
3,LLM - Based Code Generation for Querying Tempo...,IEEE Year: 2024,IEEE
4,RTLCoder: Outperforming GPT-3.5 in Design RTL ...,IEEE Year: 2024,IEEE
...,...,...,...
843,Tell Me How to Survey: Literature Review Made ...,IEEE Year: 2022,IEEE
844,Discriminative Probing and Tuning for Text-to-...,IEEE Year: 2024,IEEE
845,Persistence of RDF Data into NoSQL: A Survey a...,IEEE Year: 2022,IEEE
846,Streaming Technologies and Serialization Proto...,IEEE Year: 2024,IEEE


In [21]:
acm = pd.read_csv('ACMPapers.csv', names=['title', 'link', 'year'])
acm['source'] = 'ACM'
acm

,title,link,year,source
0,Evaluation of Code LLMs on Geospatial Code Gen...,https://dl.acm.org/doi/10.1145/3687123.3698286,January 2025,ACM
1,Teaching Code LLMs to Use Autocompletion Tools...,https://dl.acm.org/doi/10.1145/3714462,May 2024,ACM
2,CodeFuse-13B: A Pretrained Multi-lingual Code ...,https://dl.acm.org/doi/10.1145/3639477.3639719,June 2024,ACM
3,Generating Software Tests for Mobile Applicati...,https://dl.acm.org/doi/10.1145/3644032.3644454,November 2024,ACM
4,RTLFixer: Automatically Fixing RTL Syntax Erro...,https://dl.acm.org/doi/10.1145/3649329.3657353,November 2024,ACM
...,...,...,...,...
74,PromptRobust: Towards Evaluating the Robustnes...,https://dl.acm.org/doi/10.1145/3689217.3690621,November 2024,ACM
75,Towards Better Chinese Spelling Check for Sear...,https://dl.acm.org/doi/10.1145/3616855.3635847,January 2025,ACM
76,Enhancing Graph Representation Learning with W...,https://dl.acm.org/doi/10.1145/3704522.3704537,October 2024,ACM
77,Multi-Modality Co-Learning for Efficient Skele...,https://dl.acm.org/doi/10.1145/3664647.3681015,March 2024,ACM


In [22]:
sd = pd.read_csv('ScienceDirectPapers.csv', names=['title', 'link'])
sd['source'] = 'ScienceDirect'
sd

,title,link,source
0,SmartGuard: An LLM-enhanced framework for smar...,https://www.sciencedirect.com/science/article/...,ScienceDirect
1,Low-cost language models: Survey and performan...,https://www.sciencedirect.com/science/article/...,ScienceDirect
2,Multi-stage guided code generation for Large L...,https://www.sciencedirect.com/science/article/...,ScienceDirect
3,The more quality information the better: Hiera...,https://www.sciencedirect.com/science/article/...,ScienceDirect
4,MedExpQA: Multilingual benchmarking of Large L...,https://www.sciencedirect.com/science/article/...,ScienceDirect
...,...,...,...
59,Enhancing source code retrieval with joint Bi-...,https://www.sciencedirect.com/science/article/...,ScienceDirect
60,SEOSS-Queries - a software engineering dataset...,https://www.sciencedirect.com/science/article/...,ScienceDirect
61,Transformers in source code generation: A comp...,https://www.sciencedirect.com/science/article/...,ScienceDirect
62,Re-examining lexical and semantic attention: D...,https://www.sciencedirect.com/science/article/...,ScienceDirect


In [ ]:
merged_df = pd.concat([ieee, acm, sd], ignore_index=True)


In [24]:
pd.concat([t, merged_df], ignore_index=True).drop_duplicates(subset=['title'], keep='first').to_csv('merged_all_sources_no_duplicated_paper.csv', index=False)

In [1]:
import pandas as pd 
g = pd.read_csv('GoogleScholarPapers.csv', names=['title', 'link', 'year'])
g['source'] = 'Google Scholar'
ieee = pd.read_csv('IEEEPapers.csv', names=['title', 'year']) #.drop_duplicates(subset=['title'], keep='first')
ieee['source'] = 'IEEE'
acm = pd.read_csv('ACMPapers.csv', names=['title', 'link', 'year']).drop_duplicates(subset=['title'], keep='first')
acm['source'] = 'ACM'
sd = pd.read_csv('ScienceDirectPapers.csv', names=['title', 'link']).drop_duplicates(subset=['title'], keep='first')
sd['source'] = 'ScienceDirect'
merged_df = pd.concat([g, ieee, acm, sd], ignore_index=True)
merged_df.drop('year', axis=1, inplace=True)

a = pd.read_csv('survey personal - include_exclude_nodup.csv')
a[['title', 'link', 'venue', 'include/exclude', 'questions', 'future use']]
merged_df[['title', 'source']]

,title,source
0,Test-Driven Development and LLM-based Code Gen...,Google Scholar
1,[PDF] Beyond synthetic benchmarks: Assessing r...,Google Scholar
2,Classeval: A manually-crafted benchmark for ev...,Google Scholar
3,A Survey on Large Language Models for Code Gen...,Google Scholar
4,When llm-based code generation meets the softw...,Google Scholar
...,...,...
2263,Oslcfit (organic simultaneous LSTM and CNN Fit...,ScienceDirect
2264,GTFS-Madrid-Bench: A benchmark for virtual kno...,ScienceDirect
2265,EDS-MEMBED: Multi-sense embeddings based on en...,ScienceDirect
2266,SEOSS-Queries - a software engineering dataset...,ScienceDirect


In [2]:
custom_dict = {'Google Scholar': 0, 'IEEE': 1, 'ACM': 2, 'ScienceDirect': 3} 

pd.merge(merged_df[['title', 'source']], a[['title', 'link', 'venue', 'include/exclude', 'questions', 'future use']], on='title', how='inner').reindex(columns=['title', 'link', 'source', 'venue', 'include/exclude', 'questions', 'future use']).sort_values(by='source', key=lambda x: x.map(custom_dict)).to_csv('include-exclude_sheet.csv', index=False)

In [3]:
pd.merge(a[['title', 'link', 'venue', 'include/exclude', 'questions', 'future use']], merged_df[['title', 'source']], on='title', how='inner').reindex(columns=['title', 'link', 'source', 'venue', 'include/exclude', 'questions', 'future use']).sort_values(by='source', key=lambda x: x.map(custom_dict))['include/exclude'].value_counts()

include/exclude
exclude      1825
include       286
?             120
include ?      27
?               1
Name: count, dtype: int64